In [5]:
import numpy as np

Preprocessing Data

In [13]:
def problem_initialization(address, pheromone_0=0.1):
    
    with open(address, 'r') as f:
        m, n = map(int, f.readline().strip().split(" "))
        representation_matrix = np.zeros((m, n), dtype=int)
        pheromone_matrix = np.full((m, n), pheromone_0)       # pheromone initialization
        cost_vector, heuristic_vector = [], []
       
        num_costs = 0
        while num_costs < n :         # get the cost of each column
            costs = list(map(int, f.readline().strip().split(" ")))
            num_costs += len(costs)
            cost_vector.extend(costs)

        num_rows = 0
        while num_rows < m:      # fill the representation matrix
            cover_num = int(f.readline().strip())
            v = 0
            while v < cover_num:
                covers = list(map(int, f.readline().strip().split(" ")))
                for e in covers:
                    representation_matrix[num_rows][e-1] = 1
                v += len(covers)
            num_rows += 1

        # heuristic initialization
        column_sum = np.sum(representation_matrix, axis=0)
        for i in range(n):
            uncovered = m - column_sum[i]
            heuristic_vector.append(1 / (uncovered + 1))

    return cost_vector, representation_matrix, pheromone_matrix, heuristic_vector

In [17]:
cost_vec, representation_mat, pheromone_mat, heuristic_vec = problem_initialization("testcases/scp41.txt")
print(pheromone_mat[0])

[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.

In [36]:
class MMAS:

    def __init__(self, pheromone_0=0.1, cost_importance=0.7, coverage_importance=0.3, alpha=1, beta=2):
        self.num_sets = 0
        self.num_elements = 0
        self.pheromone_0 = pheromone_0
        self.cost_importance = cost_importance
        self.coverage_importance = coverage_importance
        self.alpha = alpha
        self.beta = beta
        self.representation_matrix = []
        self.heuristic_vector = []
        self.cost_vector = []
        self.pheromone_vector = []


    def initialize_problem(self, address):

        with open(address, 'r') as f:
            self.num_elements, self.num_sets = map(int, f.readline().strip().split(" "))
            self.representation_matrix = np.zeros((self.num_elements, self.num_sets), dtype=int)
            self.pheromone_vector = np.full((self.num_sets, ), self.pheromone_0)       # pheromone initialization
        
            num_costs = 0
            while num_costs < self.num_sets :         # get the cost of each column
                costs = list(map(int, f.readline().strip().split(" ")))
                num_costs += len(costs)
                self.cost_vector.extend(costs)

            num_rows = 0
            while num_rows < self.num_elements:      # fill the representation matrix
                cover_num = int(f.readline().strip())
                v = 0
                while v < cover_num:
                    covers = list(map(int, f.readline().strip().split(" ")))
                    for e in covers:
                        self.representation_matrix[num_rows][e-1] = 1
                    v += len(covers)
                num_rows += 1

            # heuristic initialization
            column_sum = np.sum(self.representation_matrix, axis=0)
            for i in range(self.num_sets):
                uncovered = self.num_elements - column_sum[i]
                self.heuristic_vector.append(1 / (uncovered + 1))

                # normalize cost and number of covered elements
                normalized_cost = self.cost_vector[i] / max(self.cost_vector)
                normalized_coverage = column_sum[i] / self.num_elements
        
            # combine factors and adjust the weights 
            self.heuristic_vector.append(self.cost_importance * normalized_cost + self.coverage_importance * normalized_coverage) 

    



In [45]:
class Ant:

    def __init__(self, mmas):
        self.colony = mmas
        self.solution = np.zeros(self.colony.num_sets, dtype=int)  


    def solution_construction(self):

        uncovered = [i for i in range(self.colony.num_elements)]

        while uncovered:
            # selection probability calculation
            selection_probabilities = np.zeros(self.colony.num_sets)
            for i in range(self.colony.num_sets):
                seti = [e for e in range(self.colony.num_elements) if self.colony.representation_matrix[e][i]]
        
                # if set i covers no uncovered elements, skip it
                if not set(seti).intersection(set(uncovered)):
                    continue
                        
                selection_probabilities[i] = (self.colony.pheromone_vector[i] ** self.colony.alpha) * (self.colony.heuristic_vector[i] ** self.colony.beta)

            # normalize probabilities
            selection_probabilities /= np.sum(selection_probabilities)
            # choose the next set to cover
            selected_set = np.random.choice(np.arange(self.colony.num_sets), p=selection_probabilities)
            # update ant's solution and uncovered elements
            self.solution[selected_set] = 1
            selectes_set_el = [e for e in range(self.colony.num_elements) if self.colony.representation_matrix[e][selected_set]]
            uncovered = set(uncovered) - set(selectes_set_el)



In [46]:
scp = MMAS()
scp.initialize_problem("testcases/scp41.txt")
ant = Ant(scp)
ant.solution_construction()
print(ant.solution)

530
970
773
722
914
838
976
181
426
578
165
938
169
234
258
219
127
347
79
450
839
646
431
288
800
111
491
472
162
623
761
290
536
343
790
624
734
474
598
321
633
241
327
40
683
139
208
222
293
594
720
677
742
932
456
419
737
60
526
149
662
443
600
275
701
267
585
363
10
952
741
998
636
943
812
894
852
587
441
655
659
631
497
617
552
549
919
588
514
622
525
618
872
221
966
752
580
250
835
801
595
541
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 

In [38]:
a = [1, 3, 5, 2]
b = [2]
set(a) - set(b)

{1, 3, 5}